In [1]:
import pandas as pd
import os
import numpyro
from refit_fvs.data.load import fia_for_diameter_growth_modeling
from refit_fvs.models.fit import fit_wykoff, fit_simpler_wykoff

In [2]:
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "0.75"
numpyro.set_platform("gpu")
numpyro.set_host_device_count(3)

# Select species
Identify species codes and the FVS variant from which bark ratio coefficients should be drawn.

In [3]:
FVS_ALPHA = "TO"
BARK_VAR = "NC"
FIA_SPCD = 631

In [4]:
BARK_COEFS = "../../data/raw/fvs_barkratio_coefs.csv"
bark = pd.read_csv(BARK_COEFS).set_index(["FIA_SPCD", "FVS_VARIANT"])

bark_b0, bark_b1, bark_b2 = bark.loc[FIA_SPCD, BARK_VAR][["BARK_B0", "BARK_B1", "BARK_B2"]]
bark_b0, bark_b1, bark_b2

(-0.26824, 0.95354, 1.0)

# Model-fitting parameters
These don't usually need to be updated when fitting a new species unless you want to add more chains or samples per chain.

In [5]:
NUM_CYCLES = 2
LOC_RANDOM = True
PLOT_RANDOM = True
CHECKPOINT_DIR = f"../../models/maicf/{FVS_ALPHA}/"
NUM_WARMUP = 1000
NUM_SAMPLES = 1000
NUM_CHAINS = 1
CHAIN_METHOD = "parallel"
NUM_BATCHES = 1
SEED = 42
PROGRESS_BAR = True
OVERWRITE = False

In [6]:
data, factors = fia_for_diameter_growth_modeling(
    path="../../data/interim/FIA_remeasured_trees_for_training.csv",
    filter_spp=[FIA_SPCD]
)
obs_variants, obs_locations, obs_plots = factors
data.describe()

,LOCATION,PLOT_ID,FIA_SPCD,MAICF,SLOPE,ASPECT,ELEV,DBH,CR,CR_NEXT,...,INC5YR,INC10YR,MEASYEAR,MEASMON,MEASYEAR_NEXT,MEASMON_NEXT,MEAS_INTERVAL,VAR_IDX,LOC_IDX,PLOT_IDX
count,2276.000000,2.276000e+03,2276.0,2276.000000,2276.000000,2276.000000,2276.000000,2276.000000,2276.000000,2276.000000,...,0.0,0.0,2276.000000,2276.000000,2276.000000,2276.000000,2276.0,2276.000000,2276.000000,2276.000000
mean,547.092267,1.701078e+11,631.0,115.305810,0.485808,2.996322,17.244288,9.547496,0.426050,0.386617,...,NaN,NaN,2004.755712,8.581283,2014.755712,7.608084,10.0,0.992531,5.612039,167.782513
std,53.653215,1.625281e+11,0.0,50.059007,0.224803,1.847743,8.161022,5.572179,0.159131,0.150486,...,NaN,NaN,2.720874,2.048421,2.720874,1.873401,0.0,0.493863,4.314373,90.244799
min,505.000000,6.010155e+10,631.0,10.000000,0.000000,0.000000,1.000000,5.000000,0.030000,0.010000,...,NaN,NaN,2001.000000,1.000000,2011.000000,4.000000,10.0,0.000000,0.000000,0.000000
25%,510.000000,6.010237e+10,631.0,76.260000,0.330000,1.308997,11.000000,6.100000,0.300000,0.300000,...,NaN,NaN,2002.000000,7.000000,2012.000000,6.000000,10.0,1.000000,0.000000,88.750000
50%,510.000000,6.010459e+10,631.0,104.540000,0.500000,2.809980,17.000000,7.500000,0.400000,0.350000,...,NaN,NaN,2004.000000,9.000000,2014.000000,8.000000,10.0,1.000000,6.000000,175.000000
75%,611.000000,4.102016e+11,631.0,142.500000,0.650000,4.625123,23.000000,10.900000,0.535000,0.500000,...,NaN,NaN,2007.000000,10.000000,2017.000000,9.000000,10.0,1.000000,10.000000,246.000000
max,710.000000,4.102034e+11,631.0,336.576478,1.550000,6.283185,39.000000,42.700000,0.900000,0.950000,...,NaN,NaN,2009.000000,12.000000,2019.000000,11.000000,10.0,3.000000,11.000000,307.000000


In [7]:
for tree_comp in ['bal', 'relht', 'ballndbh']:
    for stand_comp in ['ba', 'lnba', 'ccf']:
        for pooling in ['unpooled', 'partial']:
            model_name = f"wykoff_{pooling}_mixed_{tree_comp}-{stand_comp}"
            fit_wykoff(
                bark_b0,
                bark_b1,
                bark_b2,
                tree_comp,
                stand_comp,
                pooling,
                LOC_RANDOM,
                PLOT_RANDOM,
                NUM_CYCLES,
                data,
                model_name,
                CHECKPOINT_DIR,
                NUM_WARMUP,
                NUM_SAMPLES,
                NUM_CHAINS,
                CHAIN_METHOD,
                NUM_BATCHES,
                SEED,
                PROGRESS_BAR,
                OVERWRITE
            )

wykoff_unpooled_mixed_bal-ba
wykoff_unpooled_mixed_bal-ba_batch00


sample: 100%|█| 2000/2000 [15:54<00:00,  2.10it/s, 255 steps of size 2.01e-02. a


Done.
Samples saved at ../../models/maicf/TO//wykoff_unpooled_mixed_bal-ba_batch00.pkl
MCMC saved at ../../models/maicf/TO//wykoff_unpooled_mixed_bal-ba_batch00_mcmc.pkl
wykoff_partial_mixed_bal-ba
wykoff_partial_mixed_bal-ba_batch00


sample: 100%|█| 2000/2000 [26:43<00:00,  1.25it/s, 255 steps of size 1.66e-02. a


Done.
Samples saved at ../../models/maicf/TO//wykoff_partial_mixed_bal-ba_batch00.pkl
MCMC saved at ../../models/maicf/TO//wykoff_partial_mixed_bal-ba_batch00_mcmc.pkl
wykoff_unpooled_mixed_bal-lnba
wykoff_unpooled_mixed_bal-lnba_batch00


...

Done.
Samples saved at ../../models/maicf/TO//wykoff_partial_mixed_ballndbh-lnba_batch00.pkl
MCMC saved at ../../models/maicf/TO//wykoff_partial_mixed_ballndbh-lnba_batch00_mcmc.pkl
wykoff_unpooled_mixed_ballndbh-ccf
wykoff_unpooled_mixed_ballndbh-ccf_batch00


sample: 100%|█| 2000/2000 [16:51<00:00,  1.98it/s, 255 steps of size 1.52e-02. a


Done.
Samples saved at ../../models/maicf/TO//wykoff_unpooled_mixed_ballndbh-ccf_batch00.pkl
MCMC saved at ../../models/maicf/TO//wykoff_unpooled_mixed_ballndbh-ccf_batch00_mcmc.pkl
wykoff_partial_mixed_ballndbh-ccf
wykoff_partial_mixed_ballndbh-ccf_batch00


sample: 100%|█| 2000/2000 [26:08<00:00,  1.28it/s, 255 steps of size 1.53e-02. a


Done.
Samples saved at ../../models/maicf/TO//wykoff_partial_mixed_ballndbh-ccf_batch00.pkl
MCMC saved at ../../models/maicf/TO//wykoff_partial_mixed_ballndbh-ccf_batch00_mcmc.pkl


In [8]:
for pooling in ['unpooled', 'partial']:
    for tree_comp in ['bal', 'relht', 'ballndbh']:
        for stand_comp in ['ba', 'lnba', 'ccf']:
            model_name = f"simplerwykoff_{pooling}_mixed_{tree_comp}-{stand_comp}"
            fit_simpler_wykoff(
                bark_b0,
                bark_b1,
                bark_b2,
                tree_comp,
                stand_comp,
                pooling,
                LOC_RANDOM,
                PLOT_RANDOM,
                NUM_CYCLES,
                data,
                model_name,
                CHECKPOINT_DIR,
                NUM_WARMUP,
                NUM_SAMPLES,
                NUM_CHAINS,
                CHAIN_METHOD,
                NUM_BATCHES,
                SEED,
                PROGRESS_BAR,
                OVERWRITE
            )

simplerwykoff_unpooled_mixed_bal-ba
simplerwykoff_unpooled_mixed_bal-ba_batch00


sample: 100%|█| 2000/2000 [11:46<00:00,  2.83it/s, 255 steps of size 1.89e-02. a


Done.
Samples saved at ../../models/maicf/TO//simplerwykoff_unpooled_mixed_bal-ba_batch00.pkl
MCMC saved at ../../models/maicf/TO//simplerwykoff_unpooled_mixed_bal-ba_batch00_mcmc.pkl
simplerwykoff_unpooled_mixed_bal-lnba
simplerwykoff_unpooled_mixed_bal-lnba_batch00


sample: 100%|█| 2000/2000 [16:46<00:00,  1.99it/s, 255 steps of size 1.38e-02. a


Done.
Samples saved at ../../models/maicf/TO//simplerwykoff_unpooled_mixed_bal-lnba_batch00.pkl
MCMC saved at ../../models/maicf/TO//simplerwykoff_unpooled_mixed_bal-lnba_batch00_mcmc.pkl
simplerwykoff_unpooled_mixed_bal-ccf
simplerwykoff_unpooled_mixed_bal-ccf_batch00


...

Done.
Samples saved at ../../models/maicf/TO//simplerwykoff_partial_mixed_ballndbh-ba_batch00.pkl
MCMC saved at ../../models/maicf/TO//simplerwykoff_partial_mixed_ballndbh-ba_batch00_mcmc.pkl
simplerwykoff_partial_mixed_ballndbh-lnba
simplerwykoff_partial_mixed_ballndbh-lnba_batch00


sample: 100%|█| 2000/2000 [28:25<00:00,  1.17it/s, 511 steps of size 1.35e-02. a


Done.
Samples saved at ../../models/maicf/TO//simplerwykoff_partial_mixed_ballndbh-lnba_batch00.pkl
MCMC saved at ../../models/maicf/TO//simplerwykoff_partial_mixed_ballndbh-lnba_batch00_mcmc.pkl
simplerwykoff_partial_mixed_ballndbh-ccf
simplerwykoff_partial_mixed_ballndbh-ccf_batch00


sample: 100%|█| 2000/2000 [18:40<00:00,  1.79it/s, 255 steps of size 1.80e-02. a


Done.
Samples saved at ../../models/maicf/TO//simplerwykoff_partial_mixed_ballndbh-ccf_batch00.pkl
MCMC saved at ../../models/maicf/TO//simplerwykoff_partial_mixed_ballndbh-ccf_batch00_mcmc.pkl
